In [ ]:
import requests
import json
import time
from datetime import datetime

In [ ]:
# Set up the Azure OpenAI API endpoint and deployment details
endpoint = "https://YOUR_RESOURCE_NAME.openai.azure.com/"  
deployment = "sora"
api_key = "YOUR_API_KEY"
api_version = "2025-02-15-preview"

In [5]:
headers = {
  'Api-Key': api_key,
  'Content-Type': 'application/json',
}

In [6]:
# Set up prompt
prompt = "青い空の下、子犬が草原を駆け回る。カメラは後ろから追いかける"

In [7]:
body = {
  "prompt": prompt,
  "n_variants": "1",
  "n_seconds": "10",
  "height": "480",
  "width": "854",
  "model": deployment,
}

In [9]:
# Execute the request to generate the video
constructed_url = f"{endpoint}openai/v1/video/generations/jobs?api-version=preview"
response = requests.post(constructed_url, headers=headers, json=body)

if not response.ok:
    print("Video generation failed.")
    print(json.dumps(response.json(), sort_keys=True, indent=4, separators=(',', ': ')))
else:
    print(json.dumps(response.json(), sort_keys=True, indent=4, separators=(',', ': ')))
    response = response.json()
    job_id = response.get("id")
    status = response.get("status")
    status_url = f"{endpoint}openai/v1/video/generations/jobs/{job_id}?api-version=preview"

    print(f"Polling job status for ID: {job_id}")
    while status not in ["succeeded", "failed"]:
        time.sleep(5)
        response = requests.get(status_url, headers=headers).json()
        status = response.get("status")
        print(f"Status: {status}")

    if status == "succeeded":
        generations = response.get("generations", [])
        if generations:
            print(f"Video generation succeeded.")

            generation_id = generations[0].get("id")
            video_url = f'{endpoint}openai/v1/video/generations/{generation_id}/content/video?api-version=preview'
            video_response = requests.get(video_url, headers=headers)
            if video_response.ok:
                output_filename = "./generated/video/" + datetime.now().strftime("%Y%m%d_%H%M%S") + ".mp4"
                with open(output_filename, "wb") as file:
                    file.write(video_response.content)
                print(f'Generated video saved as "{output_filename}"')
        else:
            print("Status is succeeded, but no generations were returned.")
    elif status == "failed":
        print("Video generation failed.")
        print(json.dumps(response, sort_keys=True, indent=4, separators=(',', ': ')))

{
    "created_at": 1755151249,
    "expires_at": null,
    "failure_reason": null,
    "finished_at": null,
    "generations": [],
    "height": 480,
    "id": "task_01k2kjnbrcf1ct2k1rak3a19bg",
    "model": "sora",
    "n_seconds": 10,
    "n_variants": 1,
    "object": "video.generation.job",
    "prompt": "\u9752\u3044\u7a7a\u306e\u4e0b\u3001\u5b50\u72ac\u304c\u8349\u539f\u3092\u99c6\u3051\u56de\u308b\u3002\u30ab\u30e1\u30e9\u306f\u5f8c\u308d\u304b\u3089\u8ffd\u3044\u304b\u3051\u308b",
    "status": "preprocessing",
    "width": 854
}
Polling job status for ID: task_01k2kjnbrcf1ct2k1rak3a19bg
Status: preprocessing
Status: running
Status: running
Status: running
Status: processing
Status: processing
Status: succeeded
Video generation succeeded.
Generated video saved as "./generated/video/20250814_150220.mp4"
